In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from obspy import Stream
from pysep.utils.io import read_sem
from pysep.recsec import RecordSection

In [ ]:
write = False

In [ ]:
map = 'Brown'

# database_type = 'SymGroups'
# station = 'R33-25'
# time_length_s = 51 # corresponds to -1s to 50s of data

# database_type = 't20'
# station = 'R33-25'
# time_length_s = 41 # corresponds to -1s to 40s of data

# database_type = 't2_5'
# station = 'R33-25'
# time_length_s = 41 # corresponds to -1s to 40s of data

# database_type = 't20'
# station = 'R18-17'
# time_length_s = 31 # corresponds to -1s to 30s of data

database_type = 't2_5'
station = 'R18-17'
time_length_s = 31 # corresponds to -1s to 30s of data

component = 'Z'

In [ ]:
if database_type == 'SymGroups':
    models = ['TRIV', 'MONO_TRIV', 'MONO', 'ORTH_MONO', 'ORTH', 'TET_ORTH',
              'TET', 'XISO_TET', 'XISO', 'ISO_XISO', 'ISO',]
elif database_type == 't20':
    models  = ['t100', 't80', 't60', 't40', 't20', 't00']
elif database_type == 't2_5':
    models  = ['t20', 't17_5', 't15', 't12_5', 't10', 't7_5', 't5', 't2_5', 't00']

database_loc = f'/scratch/agupta7/specfem/rectangular_grid/{map}_{database_type}'

source_file = f'{database_loc}/{models[0]}/OUTPUT_FILES/CMTSOLUTION'
stations_file = f'{database_loc}/{models[0]}/OUTPUT_FILES/STATIONS_FILTERED'

In [ ]:
st = Stream()
for model in models:
    print(f'loading data for model: {model}')
    fid = (f'{database_loc}/{model}/OUTPUT_FILES/seismograms/XX.{station}.CX{component}.semd')
    st += read_sem(fid=fid, source=source_file, stations=stations_file)

In [ ]:
for tr in st:
    tr.differentiate()
    t1 = tr.stats.starttime
    t2 = t1 + time_length_s
    tr.trim(t1,t2)

In [ ]:
plotting_defaults = dict( st                     = st,
                          #sort_by                = 'alphabetical',
                          scale_by               = 'global_norm',                          
                          amplitude_scale_factor = 1,
                          preprocess             = False,
                          integrate              = -1,
                          max_traces_per_rs      = None,
                          xlim_s                 = [-2, time_length_s],
                          components             = component,
                          distance_units         = 'km',
                          figsize                = (6,15),                          
                          show                   = False,
                          save                   = '',
                          log_level              = 'CRITICAL', 
                          cartesian              = True,
                          synsyn                 = True,
                          linewidth              = 0.5,
                          title                  = '',
                          y_label_loc            = 'x_min',
                          y_label_fontsize       = 0,
                          azimuth_bin_c          = 'None' )

In [ ]:
rs = RecordSection(**plotting_defaults)
rs.run()
plt.grid(False, which='both')
if write: plt.savefig(f'record_section_{database_type}_{station}.png')
plt.show()